# Information Retrieval - Spring 2022
- topic: **Saadi's poems**
- members: _Javad Hezareh, Yasin Moosavi, Amirali Ebrahimzadeh_

---

In this project we've implemented a retrieval system for saadi's poems.

We have crawled our data from 'ganjoor.net' and implemented 4 method to retrieve similar poems such as boolean, tf-idf, transformers and using fasttext.

---

Contents:
1. Crawling
2. Data Explenation
3. Load Data
4. Preprocessing
5. Utils
6. Methodes
    - Boolean
    - TF-IDF
    - Transformer
    - Fast Text


In [4]:
# change this to 'True' if you want to use cralwer
# note: colab can't connect to 'ganjoor.net', so run this notebook localy in case of
# useing crawler
crawl = False

# Crawling

In [ ]:
!pip install bs4

In [ ]:
import requests
import json
from bs4 import BeautifulSoup

In [ ]:
ganjoor = 'https://ganjoor.net'
bostan = '/saadi/boostan'
golestan = '/saadi/golestan'

In [ ]:
def extract_poems(source):
    data = {}
    data['name'] = source.split('/')[-1]
    data['url'] = source
    
    tag = 'div.part-title-block > a'
    
    response = requests.get(source)
    soup = BeautifulSoup(response.text, 'lxml')
    
    babs = []
    for p in soup.select(tag):
        bab = {}
        name = p.get_text()
        url = p.attrs.get('href')
        bab['bab-name'] = name
        bab['bab-url'] = url
        
        new_url = f'https://ganjoor.net{url}'
        bab['bab-sections'] = get_section_data(new_url)
        babs.append(bab)
        
        print(f"bab '{name}' is done.")
    
    data['babs'] = babs
    return data


def get_section_data(url):
    secs = []
    
    tag = 'div.poem p.poem-excerpt > a'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    
    sections = list(soup.select(tag))
    if len(sections) == 0:
        section = {}
        section['sec-name'] = 'شعر'
        section['sec-poems'] = get_poems(url)
        
        secs.append(section)
    else:
        for s in sections:
            section = {}
            name = s.get_text()
            url = s.attrs.get('href')
            
            section['sec-name'] = name
            
            new_url = f'https://ganjoor.net{url}'
            section['sec-poems'] = get_poems(new_url)

            secs.append(section)

    return secs


def get_poems(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    
    poems = []
    couplet = soup.find_all('div', {'class': ['n', 'b']})
    for c in couplet:
        mesra = [x.get_text() for x in c.find_all('p')]
        poems.append('\t\t'.join(mesra))
    return poems

In [ ]:
sources = [ganjoor + bostan, ganjoor + golestan]
data = []

if crawl:
    # crawl data from ganjoor.net
    for addr in sources:
        data.append(extract_poems(addr))
        print()

    # save data in file
    for s in range(2):
        file_name = ['boostan', 'golestan'][s]
        with open(f'{file_name}.json', 'w', encoding='utf8') as jf:
            json.dump(data[s], jf, ensure_ascii=False)

bab 'در نیایش خداوند' is done.
bab 'باب اول در عدل و تدبیر و رای' is done.
bab 'باب دوم در احسان' is done.
bab 'باب سوم در عشق و مستی و شور' is done.
bab 'باب چهارم در تواضع' is done.
bab 'باب پنجم در رضا' is done.
bab 'باب ششم در قناعت' is done.
bab 'باب هفتم در عالم تربیت' is done.
bab 'باب هشتم در شکر بر عافیت' is done.
bab 'باب نهم در توبه و راه صواب' is done.
bab 'باب دهم در مناجات و ختم کتاب' is done.

bab 'دیباچه' is done.
bab 'باب اول در سیرت پادشاهان' is done.
bab 'باب دوم در اخلاق درویشان' is done.
bab 'باب سوم در فضیلت قناعت' is done.
bab 'باب چهارم در فواید خاموشی' is done.
bab 'باب پنجم در عشق و جوانی' is done.
bab 'باب ششم در ضعف و پیری' is done.
bab 'باب هفتم در تأثیر تربیت' is done.
bab 'باب هشتم در آداب صحبت' is done.



# Data Explenation

each _.json_ file consists of Saadi's poems from his books. Each book consists of different _bab_s and each bab consists of different _section_s and in sections we can accesse poems

|books field|babs field|sections field|
|:---:|:---:|:---:|
|name|bab-name|sec-name|
|url|bab-url|sec-poems|
|babs|bab-sections||

# Load Data

In [ ]:
# uncomment below codes if you don't have data
# !wget --no-cache --backups=1 https://github.com/MJavadHzr/Saadi_Poems/blob/master/stop_words.txt
# !wget --no-cache --backups=1 https://github.com/MJavadHzr/Saadi_Poems/blob/master/boostan.json
# !wget --no-cache --backups=1 https://github.com/MJavadHzr/Saadi_Poems/blob/master/golestan.json
# !wget --no-cache --backups=1 https://github.com/MJavadHzr/Saadi_Poems/blob/master/test_inputs.txt

In [ ]:
import json

boostan = 'boostan'
golestan = 'golestan'

In [ ]:
# load data
with open(f'{boostan}.json', encoding='utf-8') as fh:
    boostan = json.load(fh)
    
with open(f'{golestan}.json', encoding='utf-8') as fh:
    golestan = json.load(fh)
    
data = [boostan, golestan]

# Preprocessing

In [ ]:
!pip install hazm

In [ ]:
from hazm import *
import copy
import codecs
import string

In [ ]:
class Preprocessor:
    def __init__(self, stopwords=True, punctuation=True):
        self.normalizer = Normalizer()
        self.lemmatizer = Lemmatizer()
        self.tokenizer = word_tokenize
        self.stopwords = [self.normalizer.normalize(x.strip()) for x in codecs.open('stop_words.txt','r','utf-8').readlines()] if stopwords else ''
        # stopwords = ['از', 'به', 'در', 'با', 'برای', 'تا', 'چون', 'آن', 'این', 'همان', 'همین', 'به', 'بی', 'است', 'نیست', '', '']
        self.punctuation = ''.join(['،', '؛', '؟']) + string.punctuation if punctuation else ''
        
    
    def _normalize(self, sentence, normalizer, lemmatizer, tokenizer, punctuation, stopwords, tokenize):
        s = normalizer.normalize(sentence)
        s = s.replace('\u200c', ' ')
        
        tokens = tokenizer(s)
        
        tokens = [t for t in tokens if t not in stopwords]
        tokens = [t for t in tokens if t not in punctuation]
        
        tokens = [lemmatizer.lemmatize(t) for t in tokens]
        
        if tokenize:
            return tokens
        else:
            return ' '.join(tokens)
    
    def transform(self, sentence, tokenize):
        return self._normalize(sentence, self.normalizer, self.lemmatizer, self.tokenizer, self.stopwords, self.punctuation, tokenize)

    def fit(self, database, tokenize):
        norm_func = lambda x: self._normalize(x, self.normalizer, self.lemmatizer, self.tokenizer, self.stopwords, self.punctuation, tokenize)
        norm_data = copy.deepcopy(database)
        # normalize
        for d in norm_data:
            for b in d['babs']:
                for s in b['bab-sections']:
                    s['sec-poems'] = list(map(norm_func, s['sec-poems']))
        
        return norm_data

In [ ]:
preprocessor = Preprocessor()

# Utils

In [ ]:
def get_couplets(database):
    all_couplets = []
    for d in database:
        for b in d['babs']:
            for s in b['bab-sections']:
                all_couplets += s['sec-poems']
    return all_couplets


def get_poems(database):
    all_poems = []
    for d in database:
        for b in d['babs']:
            for s in b['bab-sections']:
                all_poems.append(s['sec-poems'])
    return all_poems


def get_tokens(database):
    all_couplets = get_couplets(database)
    tokens = set()
    for couplet in all_couplets:
        tokens = tokens.union(couplet)
    return tokens


def test_model(model, name):
    f = open('test_inputs.txt', 'r', encoding='utf8')
    inputs = f.read()
    f.close()

    results = []
    f = open(f'{name}_result.txt', 'w', encoding='utf8')
    for query in inputs.split('\n'):
        results = []
        out = model.search(query)
        for o in out:
            results.append(o['key'])
        print(out)
        out = '\n'.join(results)
        f.write(out)
        f.write('\n\n')
    f.close()

# Boolean

In [ ]:
import numpy as np
import copy

class BooleanModel:
    def __init__(self, database, preprocessor):
        self.database = database
        self.preprocessor = preprocessor
        self.norm_data = self.preprocessor.fit(database, tokenize=True)


    def fit(self):
        self.vec_dic = {}
        self.terms = get_tokens(self.norm_data)
        docs = [' '.join(x) for x in get_couplets(self.norm_data)]
        raw_docs = get_couplets(self.database)

        for i in range(len(docs)):
            doc_tok = docs[i].split()
            attendance = []
            for term in self.terms:
                if term in doc_tok:
                    attendance.append(1)
                else:
                    attendance.append(0)
    
            cp = copy.copy(attendance)
            self.vec_dic.update({raw_docs[i]: cp})


    def search(self, query, method='couplet', k=10):
        qvec = self._build_query_vec(query)

        dictionary = {}
        for i in self.vec_dic.keys():
            dictionary.update({i: np.dot(qvec, self.vec_dic[i])})
    
        ls = sorted(dictionary, key=dictionary.get, reverse=True)[:k]
        out = []
        for i in ls:
            out.append({ "key" : i , "value" : dictionary[i]})
        
        return out


    def _build_query_vec(self, query):
        query = self.preprocessor.transform(query, tokenize=True)
        
        qvec = []
        for i in self.terms:
            if i in query:
                qvec.append(1)
            else:
                qvec.append(0)
    
        return qvec

In [ ]:
boolean_model = BooleanModel(data, preprocessor)
boolean_model.fit()

### Boolean Testing

In [ ]:
query = 'دلم خانه‌ی مهر یار است و بس از آن می‌نگنجد در او کین کس'
boolean_model.search(query)

[{'key': 'دلم خانهٔ مهر یار است و بس\t\tاز آن می\u200cنگنجد در او کین کس',
  'value': 5},
 {'key': 'که گردد درونش به کین تو ریش\t\tچو یاد آیدش مهر پیوند خویش',
  'value': 2},
 {'key': 'زرش داد و اسب و قبا پوستین\t\tچه نیکو بود مهر در وقت کین',
  'value': 2},
 {'key': 'به حکم ضرورت در پی کاروانی افتاد و برفت. شبانگه برسیدند به مقامی که از دزدان پر خطر بود. کاروانیان را دید لرزه بر اندام اوفتاده و دل بر هلاک نهاده. گفت: اندیشه مدارید که یکی منم در این میان که به تنها پنجاه مرد را جواب دهم و دیگر جوانان هم یاری کنند. این بگفت و مردم کاروان را به لاف او دل قوی گشت و به صحبتش شادمانی کردند و به زاد و آبش دستگیری واجب دانستند. جوان را آتش معده بالا گرفته بود و عنان طاقت از دست رفته. لقمه ای چند از سر اشتها تناول کرد و دمی چند آب در سرش آشامید تا دیو درونش بیارمید و بخفت. پیرمردی جهاندیده در آن میان بود. گفت: ای یاران! من از این بدرقه شما اندیشناکم، نه چندان که از دزدان؛ چنان که حکایت کنند که عربی را درمی چند گرد آمده بود و به شب از تشویش لوریان در خانه تنها خوابش نمی برد. یکی را از دوستان پی

In [ ]:
test_model(boolean_model, 'boolean')

[{'key': 'تقصیر و تقاعدی که در مواظبت خدمت بارگاه خداوندی می\u200cرود بنا بر آن است که طایفه\u200cای از حکماء هندوستان در فضایل بزرجمهر سخن می\u200cگفتند، به آخر جز این عیبش ندانستند که در سخن گفتن بطیء است یعنی درنگ بسیار می\u200cکند و مستمع را بسی منتظر باید بودن تا تقریر سخنی کند. بزرجمهر بشنید و گفت: اندیشه کردن که چه گویم به از پشیمانی خوردن که چرا گفتم.', 'value': 2}, {'key': 'بر سلامت حالش شادمانی کرد و از هر دری سخن گفتند تا ملک به انجام سخن گفت: چنین که من این هر دو طایفه را دوست دارم در جهان کس ندارد یکی علما و دیگر زهاد را.', 'value': 2}, {'key': 'اما صاحب دنیا به عین عنایت حق ملحوظ است و به حلال از حرام محفوظ. من همانا که تقریر این سخن نکردم و برهان بیان نیاوردم، انصاف از تو توقع دارم هرگز دیده\u200cای دست دعایی بر کتف بسته یا بینوایی به زندان در نشسته یا پردهٔ معصومی دریده یا کفی از معصم بریده، الا به علت درویشی؟ شیرمردان را به حکم ضرورت در نقبها گرفته\u200cاند و کعبها سفته و محتمل است آن که یکی را از درویشان نفس اماره طلب کند، چو قوت احصانش نباشد، به عصیان مبتلا گردد که ب

MRR for this test: 0.9316

# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import itertools


class TFIDFModel:
    def __init__(self, database, preprocessor):
        self.database = database
        self.preprocessor = preprocessor
        self.normal_couplets = get_couplets(self.preprocessor.fit(database, tokenize=False))

    
    def fit(self):
        self.cv = CountVectorizer()
        doc_term = self.cv.fit_transform(self.normal_couplets)
        
        idfs = TfidfTransformer()
        idfs.fit(doc_term)
        self.tf_idfs = idfs.transform(doc_term)


    def search(self, query, method='couplet', k=10):
        query = [self.preprocessor.transform(query, tokenize=False)]
        query_term = self.cv.transform(query)
        similarity = cosine_similarity(self.tf_idfs, query_term)

        all_couplets = get_couplets(self.database)
        results = list(itertools.chain.from_iterable(similarity))
        indices = sorted(range(len(results)), key=lambda i: results[i], reverse=True)[:k]
        dic =[]
        for i in indices:
            dic.append({"key": all_couplets[i], "value": results[i]})

        return dic

In [ ]:
tfidf_model = TFIDFModel(data, preprocessor)
tfidf_model.fit()

### TF-IDF Testing


In [ ]:
query = 'دلم خانه‌ی مهر یار است و بس از آن می‌نگنجد در او کین کس'
tfidf_model.search(query)

[{'key': 'دلم خانهٔ مهر یار است و بس\t\tاز آن می\u200cنگنجد در او کین کس',
  'value': 0.9878470873757951},
 {'key': 'زرش داد و اسب و قبا پوستین\t\tچه نیکو بود مهر در وقت کین',
  'value': 0.3338621953720987},
 {'key': 'که گردد درونش به کین تو ریش\t\tچو یاد آیدش مهر پیوند خویش',
  'value': 0.33358754735404306},
 {'key': 'فرستاده را داد مهری درم\t\tکه مهر است بر نام حاتم کرم',
  'value': 0.3133654293690166},
 {'key': 'خرابت کند شاهد خانه کن\t\tبرو خانه آباد گردان به زن',
  'value': 0.28767540043785017},
 {'key': 'یکی از لوازم صحبت آن است که خانه بپردازی یا با خانه خدای در سازی.',
  'value': 0.25674008165603646},
 {'key': 'کز او داد مظلوم مسکین او\t\tبخواهند و از دیگران کین او',
  'value': 0.2532078240723843},
 {'key': 'چو بینی که یاران نباشند یار\t\tهزیمت ز میدان غنیمت شمار',
  'value': 0.22940066724499325},
 {'key': 'نه آن می\u200cکند یار در شاهدی\t\tکه با او توان گفتن از زاهدی',
  'value': 0.21742898915360373},
 {'key': 'یکی گفتا: چگونه\u200cای در مفارقت یار عزیز؟',
  'value': 0.2088814

In [ ]:
test_model(tfidf_model, 'tfidf')

[{'key': 'بر سلامت حالش شادمانی کرد و از هر دری سخن گفتند تا ملک به انجام سخن گفت: چنین که من این هر دو طایفه را دوست دارم در جهان کس ندارد یکی علما و دیگر زهاد را.', 'value': 0.3046359197677323}, {'key': 'بسیج ِ سخن گفتن آنگاه کن\t\tکه دانی که در کار گیرد سخُن', 'value': 0.2981004720289609}, {'key': 'سخن را سر است اى خردمند و بن\t\tمیاور سخن در میان سخن', 'value': 0.29560937712607077}, {'key': 'سوم باب عشق است و مستی و شور\t\tنه عشقی که بندند بر خود بزور', 'value': 0.2779499225971604}, {'key': 'سخنهای دانای شیرین سخن\t\tگرفت اندر آن هر دو شمشاد بن', 'value': 0.2575933548834544}, {'key': 'سخن ماند از عاقلان یادگار\t\tز سعدی همین یک سخن یاد دار', 'value': 0.24185066100861585}, {'key': 'تو که در بند خویشتن باشی\t\tعشق باز دروغ زن باشی', 'value': 0.24182026522259759}, {'key': 'اگر مرد عشقی کم خویش گیر\t\tو گر نه ره عافیت پیش گیر', 'value': 0.23397951904374173}, {'key': 'تقصیر و تقاعدی که در مواظبت خدمت بارگاه خداوندی می\u200cرود بنا بر آن است که طایفه\u200cای از حکماء هندوستان در فضایل بز

MRR for this test: 0.9444

# Transformer

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models
from sentence_transformers import util
import numpy as np
import torch

In [ ]:
class ParsTransformer:
    def __init__(self, database, preprocessor, device):
        self.database = database
        self.norm_data = preprocessor.fit(database, tokenize=False)
        self.preprocessor = preprocessor
        self.device = device

    
    def fit(self):
        word_embedding_model = models.Transformer('HooshvareLab/bert-fa-zwnj-base', max_seq_length=256)
        pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

        self.model = SentenceTransformer(modules=[word_embedding_model, pooling_model], device=self.device)

        couplets = get_couplets(self.norm_data)
        self.encodes = self.model.encode(couplets)

    
    def search(self, query, method='couplet', k=10):
        query = self.preprocessor.transform(query, tokenize=False)
        
        q_encode = self.model.encode(query)

        similarity = util.cos_sim(q_encode, self.encodes).numpy()[0]
        indices = np.argsort(similarity)[::-1][:k]

        result = []
        raw_couplets = get_couplets(self.database)
        for idx in indices:
            result.append({'key': raw_couplets[idx], 'value': similarity[idx]})
        
        return result

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

transformer_model = ParsTransformer(data, preprocessor, device)
transformer_model.fit()

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.poo

### Transformer Testing

In [ ]:
query = 'دلم خانه‌ی مهر یار است و بس از آن می‌نگنجد در او کین کس'
transformer_model.search(query)

[{'key': 'دلم خانهٔ مهر یار است و بس\t\tاز آن می\u200cنگنجد در او کین کس',
  'value': 1.0000001},
 {'key': 'دو صاحبدل نگه دارند مویی\t\tهمیدون سرکشی و آزرم جویی',
  'value': 0.7621424},
 {'key': 'بگفتا همی\u200cگریم از روزگار\t\tکه طفلان بیچاره دارم چهار',
  'value': 0.7602873},
 {'key': 'نشاید به دستان شدن در بهشت\t\tکه بازت رود چادر از روی زشت',
  'value': 0.75739247},
 {'key': 'مگر دل نهادی به مردن ز پس\t\tکه بر می\u200cنخیزی به بانگ جرس؟',
  'value': 0.75179154},
 {'key': 'نیوشنده شد زین سخن تنگدل\t\tبه فکرت فرو رفت چون خر به گل',
  'value': 0.74671483},
 {'key': 'چو بلبل، سرایان چو گل تازه روی\t\tز شوخی در افکنده غلغل به کوی',
  'value': 0.7455336},
 {'key': 'اگر تنگدستی مرو پیش یار\t\tوگر سیم داری بیا و بیار',
  'value': 0.7452309},
 {'key': 'پریشان شود گل به باد سحر\t\tنه هیزم که نشکافدش جز تبر',
  'value': 0.74438655},
 {'key': 'رزی داشتم بر در خانه، گفت\t\tبه سایه درش نیکمردی بخفت',
  'value': 0.7413236}]

In [ ]:
test_model(transformer_model, 'transformer')

[{'key': 'شنید این سخن پیر فرخنده فال\t\tسخندان بود مرد دیرینه سال', 'value': 0.74390304}, {'key': 'چو مردم سخن گفت باید به هوش\t\tوگر نه شدن چون بهایم خموش', 'value': 0.73375636}, {'key': 'اگر چون زنان جست خواهی گریز\t\tمرو آب مردان جنگی مریز', 'value': 0.7316015}, {'key': 'گر چه دانی که نشنوند بگوی\t\tهر چه دانی ز نیکخواهی و پند', 'value': 0.73033625}, {'key': 'ای به ناموس کرده جامه سپید\t\tبهر پندار خلق و نامه سیاه', 'value': 0.722579}, {'key': 'سخن میان دو دشمن چنان گوی که گر دوست گردند شرم زده نشوی.', 'value': 0.7222427}, {'key': 'به بد گفتن خلق چون دم زدی\t\tاگر راست گویی سخن هم بدی', 'value': 0.71363235}, {'key': 'چنین گفت پیری پسندیده هوش\t\tخوش آید سخنهای پیران به گوش', 'value': 0.710756}, {'key': 'شنیدم که می\u200cگفت و گردن به بند\t\tنباشد حذر با قدر سودمند', 'value': 0.7088059}, {'key': 'گر راست سخن گویی و در بند بمانی\t\tبه زآن که دروغت دهد از بند رهایی', 'value': 0.70848775}]
[{'key': 'طلبکار خیر است امیدوار\t\tخدایا امیدی که دارد برآر', 'value': 0.7731704}, {'key': 'بسی 

MRR for this test: 0.9583

# FastText

In [ ]:
!pip install fasttext

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from functools import reduce
import numpy as np
import fasttext

In [ ]:
class FastText:
    def __init__(self, database, preprocessor, wordNgrams):
        self.database = database
        self.norm_data = preprocessor.fit(database, tokenize=True)
        self.preprocessor = preprocessor
        self.wordNgrams = wordNgrams


    def fit(self):
        corpus = [' '.join(x) for x in get_couplets(self.norm_data)]
        content = '\n'.join(corpus)

        addr = 'fasttext.txt'
        f = open(addr, 'w', encoding='utf8')
        f.write(content)
        self.model = fasttext.train_unsupervised(addr)

        # save idfs for averaging
        vectorizer = TfidfVectorizer()
        vectorizer.fit(corpus)
        self.idfs = vectorizer.idf_
        self.term_idf = vectorizer.vocabulary_
        

    def search(self, query, method='couplet', k=10):
        if method == 'couplet':
            return self._search_couplet(query, k)
        else:
            return self._search_poem(query, k)


    def _search_couplet(self, query, k):
        query = self.preprocessor.transform(query, tokenize=True)
        data = get_couplets(self.norm_data)
        similarity = np.array([self._similarity(query, couplet) for couplet in data])
        indices = np.argsort(similarity)[::-1][:k]

        result = []
        couplets = get_couplets(self.database)
        for idx in indices:
            result.append({ "key" : couplets[idx] , "value" : similarity[idx]})
        
        return result


    def _search_poem(self, query, k):
        pass


    def _doc2vec(self, sec):
        if len(sec) == 0:
            return np.zeros((300,))
        
        coeff = []
        vecs = np.zeros((100,))

        for i in range(len(sec)):
            new_vec = self.model.get_word_vector(sec[i])
            vecs = np.vstack((vecs, new_vec))
            try:
                coeff.append(self.idfs[self.term_idf[sec[i]]])
            except:
                coeff.append(1)

        vecs = vecs[1:]
        maxes = np.amax(vecs, axis=0)
        mins = np.amin(vecs, axis=0)
        avgs = np.average(vecs, axis=0, weights=coeff)

        result = np.hstack((maxes, mins))
        result = np.hstack((result, avgs))
        norm = np.linalg.norm(result)
        if norm != 0:
            result /= norm
        return result


    def _similarity(self, d1, d2):
        v1 = self._doc2vec(d1)
        v2 = self._doc2vec(d2)

        return v1 @ v2

In [ ]:
fasttext_model = FastText(data, preprocessor, 5)
fasttext_model.fit()

## FastText testing

In [ ]:
query = 'دلم خانه‌ی مهر یار است و بس از آن می‌نگنجد در او کین کس'
fasttext_model.search(query)

[{'key': 'دلم خانهٔ مهر یار است و بس\t\tاز آن می\u200cنگنجد در او کین کس',
  'value': 1.0},
 {'key': 'معانی است در زیر حرف سیاه\t\tچو در پرده معشوق و در میغ ماه',
  'value': 0.9999722822678931},
 {'key': 'ور آوازه خواهی در اقلیم فاش\t\tبرون حله کن گو درون حشو باش',
  'value': 0.9999662063391745},
 {'key': 'به زیر آمد از غرفه خلوت نشین\t\tبه پایش در افتاد سر بر زمین',
  'value': 0.9999618037946284},
 {'key': 'انگشت تعجب جهانی\t\tاز گفت و شنید ما به دندان',
  'value': 0.9999565421454938},
 {'key': 'بصر در سر و فکر و رای و تمیز\t\tجوارح به دل، دل به دانش عزیز',
  'value': 0.9999565321022973},
 {'key': 'سکندر که بر عالمی حکم داشت\t\tدر آن دم که بگذشت و عالم گذاشت',
  'value': 0.9999537504375204},
 {'key': 'پسر کاو میان قلندر نشست\t\tپدر گو ز خیرش فرو شوی دست',
  'value': 0.9999524692972396},
 {'key': 'چه دانند جیحونیان قدر آب\t\tز واماندگان پرس در آفتاب',
  'value': 0.9999473666770833},
 {'key': 'چو بلبل، سرایان چو گل تازه روی\t\tز شوخی در افکنده غلغل به کوی',
  'value': 0.9999433486746243

In [ ]:
test_model(fasttext_model, 'fasttext')

[{'key': 'زورمندی مکن بر اهل زمین\t\tتا دعایی بر آسمان نرود', 'value': 0.9999827534267595}, {'key': 'یکی زندگانی تلف کرده بود\t\tبه جهل و ضلالت سر آورده بود', 'value': 0.9999804540631531}, {'key': 'سگالند از او نیکمردان حذر\t\tکه خشم خدای است بیدادگر', 'value': 0.9999772640593255}, {'key': 'در اندیشه\u200cام تا کدامم کریم\t\tاز آن سنگدل دست گیرد به سیم', 'value': 0.9999741457778801}, {'key': 'قدیمی نکوکار نیکی\u200cپسند\t\tبه کلک قضا در رحم نقش\u200cبند', 'value': 0.999972896389823}, {'key': 'یکی زآن میان به فراست به جای آورد و گفت: ای ملک! ما در این دنیا به جیش از تو کمتریم و به عیش خوشتر و به مرگ برابر و به قیامت بهتر.', 'value': 0.9999721450121442}, {'key': 'چو چنگش کشیدند حالی به موی\t\tغلامان و چون دف زدندش به روی', 'value': 0.9999695807660721}, {'key': 'چو زاو کرده باشم تحمل بسی\t\tتوانم جفا بردن از هر کسی', 'value': 0.999969269611994}, {'key': 'که این را ندانم چه خوانند و کیست!\t\tنخواهد به سامان در این ملک زیست', 'value': 0.9999689696333075}, {'key': 'که تا هست حاتم در ایام من\

MRR for this test: 0.5571